In [1]:
## This file implements neural networks and logistic regression on pyopresabsSTCC_quant.
## Due to the imbalanced data, we implement the over-sampling method and the combination of 
## over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 84.87% for combination data and 81.12% for over-sampling data.
## To improve neural network performance, we implement lasso regression to select important features.
## However, with the new dataset, the accuracy becomes 83.93% for combination, and 80.42% for over-sampling data.
## For logistic regression, the accuracy is 98.32% for combination data and 99.3% for over-sampling data.
## For random forest, the accuracy is 98.32% for combination, and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.27% for combination data, and 98.79% for over-sampling.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/pyopresabsSTCC_quant.csv')
df.shape

(255, 541)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.158375
1      0.157875
2      0.310875
3      0.160125
4      0.160500
5      0.205000
6      0.158250
7      0.158375
8      0.166000
9      0.157750
10     0.159375
11     0.159000
12     0.161000
13     0.161875
14     0.157875
15     0.157125
16     0.305000
17     0.281250
18     0.244625
19     0.156875
20     0.169625
21     0.282500
22     0.156750
23     0.161875
24     0.525750
25     0.155375
26     0.171375
27     0.156500
28     0.158625
29     0.288500
         ...   
225    0.179000
226    0.333889
227    0.400222
228    0.499444
229    0.523000
230    0.267250
231    0.209000
232    0.243625
233    0.201875
234    0.160000
235    0.184250
236    0.178000
237    0.157000
238    0.158250
239    0.201000
240    0.232750
241    0.174250
242    0.159250
243    0.160125
244    0.154000
245    0.156875
246    0.233375
247    0.157375
248    0.158375
249    0.158625
250    0.231250
251    0.174625
252    0.157000
253    0.167625
254    0.163000
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno,ST,CC
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.158375,5,5
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0.157875,8,8
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.310875,5,5
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.160125,5,5
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0.160500,5,5


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 541)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 540)

In [13]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno,ST,CC
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,8,8
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,5,5


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 539) (255,)


In [15]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 192), (1, 202)]


In [16]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [17]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [18]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [19]:
############# Fully-Connected Neural Network ################

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [21]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [22]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 275 samples, validate on 119 samples
Epoch 1/1000
275/275 [==============================] - 0s 507us/step - loss: 3.7530 - accuracy: 0.5382 - val_loss: 0.6327 - val_accuracy: 0.4874
Epoch 2/1000
275/275 [==============================] - 0s 98us/step - loss: 0.6380 - accuracy: 0.5273 - val_loss: 0.6188 - val_accuracy: 0.6555
Epoch 3/1000
275/275 [==============================] - 0s 119us/step - loss: 0.6207 - accuracy: 0.6073 - val_loss: 0.6048 - val_accuracy: 0.6807
Epoch 4/1000
275/275 [==============================] - 0s 91us/step - loss: 0.6093 - accuracy: 0.6436 - val_loss: 0.5961 - val_accuracy: 0.7227
Epoch 5/1000
275/275 [==============================] - 0s 189us/step - loss: 0.6054 - accuracy: 0.6691 - val_loss: 0.5864 - val_accuracy: 0.7479
Epoch 6/1000
275/275 [==============================] - 0s 153us/step - loss: 0.5957 - accuracy: 0.6945 - val_loss: 0.5780 - val_accuracy: 0.7395
Epoch 7/1000
275/275 [==============================] - 0s 149us/step - loss: 0.

275/275 [==============================] - 0s 141us/step - loss: 0.4035 - accuracy: 0.8582 - val_loss: 0.3823 - val_accuracy: 0.8739
Epoch 57/1000
275/275 [==============================] - 0s 128us/step - loss: 0.3958 - accuracy: 0.8400 - val_loss: 0.3913 - val_accuracy: 0.8487
Epoch 58/1000
275/275 [==============================] - 0s 122us/step - loss: 0.3959 - accuracy: 0.8400 - val_loss: 0.3962 - val_accuracy: 0.8319
Epoch 59/1000
275/275 [==============================] - 0s 119us/step - loss: 0.4093 - accuracy: 0.8182 - val_loss: 0.3798 - val_accuracy: 0.8487
Epoch 60/1000
275/275 [==============================] - 0s 132us/step - loss: 0.3910 - accuracy: 0.8436 - val_loss: 0.3752 - val_accuracy: 0.8908
Epoch 61/1000
275/275 [==============================] - 0s 126us/step - loss: 0.3894 - accuracy: 0.8473 - val_loss: 0.3621 - val_accuracy: 0.9160
Epoch 62/1000
275/275 [==============================] - 0s 208us/step - loss: 0.4000 - accuracy: 0.8364 - val_loss: 0.3726 - val_ac

Epoch 112/1000
275/275 [==============================] - 0s 122us/step - loss: 0.2986 - accuracy: 0.8582 - val_loss: 0.3138 - val_accuracy: 0.8571
Epoch 113/1000
275/275 [==============================] - 0s 111us/step - loss: 0.3153 - accuracy: 0.8436 - val_loss: 0.2903 - val_accuracy: 0.9160
Epoch 114/1000
275/275 [==============================] - 0s 133us/step - loss: 0.3545 - accuracy: 0.8291 - val_loss: 0.2884 - val_accuracy: 0.8992
Epoch 115/1000
275/275 [==============================] - 0s 139us/step - loss: 0.2922 - accuracy: 0.8582 - val_loss: 0.3137 - val_accuracy: 0.8571
Epoch 116/1000
275/275 [==============================] - 0s 169us/step - loss: 0.2902 - accuracy: 0.8582 - val_loss: 0.2961 - val_accuracy: 0.8739
Epoch 117/1000
275/275 [==============================] - 0s 140us/step - loss: 0.3026 - accuracy: 0.8473 - val_loss: 0.3123 - val_accuracy: 0.8908
Epoch 118/1000
275/275 [==============================] - 0s 125us/step - loss: 0.3165 - accuracy: 0.8327 - val_

275/275 [==============================] - 0s 220us/step - loss: 0.2538 - accuracy: 0.8509 - val_loss: 0.2850 - val_accuracy: 0.8571
Epoch 168/1000
275/275 [==============================] - 0s 137us/step - loss: 0.2585 - accuracy: 0.8618 - val_loss: 0.2821 - val_accuracy: 0.8571
Epoch 169/1000
275/275 [==============================] - 0s 88us/step - loss: 0.2629 - accuracy: 0.8582 - val_loss: 0.2533 - val_accuracy: 0.8824
Epoch 170/1000
275/275 [==============================] - 0s 132us/step - loss: 0.2775 - accuracy: 0.8691 - val_loss: 0.2588 - val_accuracy: 0.9160
Epoch 171/1000
275/275 [==============================] - 0s 117us/step - loss: 0.2493 - accuracy: 0.8582 - val_loss: 0.2482 - val_accuracy: 0.9160
Epoch 172/1000
275/275 [==============================] - 0s 114us/step - loss: 0.2675 - accuracy: 0.8364 - val_loss: 0.2714 - val_accuracy: 0.8571
Epoch 173/1000
275/275 [==============================] - 0s 107us/step - loss: 0.2516 - accuracy: 0.8655 - val_loss: 0.2825 - v

275/275 [==============================] - 0s 109us/step - loss: 0.2371 - accuracy: 0.8655 - val_loss: 0.2370 - val_accuracy: 0.8992
Epoch 223/1000
275/275 [==============================] - 0s 100us/step - loss: 0.2317 - accuracy: 0.8473 - val_loss: 0.2586 - val_accuracy: 0.8487
Epoch 224/1000
275/275 [==============================] - 0s 95us/step - loss: 0.2545 - accuracy: 0.8436 - val_loss: 0.2453 - val_accuracy: 0.8739
Epoch 225/1000
275/275 [==============================] - 0s 94us/step - loss: 0.2545 - accuracy: 0.8436 - val_loss: 0.2508 - val_accuracy: 0.8739
Epoch 226/1000
275/275 [==============================] - 0s 96us/step - loss: 0.2425 - accuracy: 0.8691 - val_loss: 0.2404 - val_accuracy: 0.8739
Epoch 227/1000
275/275 [==============================] - 0s 96us/step - loss: 0.2275 - accuracy: 0.8691 - val_loss: 0.2629 - val_accuracy: 0.8487
Epoch 228/1000
275/275 [==============================] - 0s 93us/step - loss: 0.2512 - accuracy: 0.8400 - val_loss: 0.2659 - val_a

...

275/275 [==============================] - 0s 84us/step - loss: 0.2319 - accuracy: 0.8582 - val_loss: 0.2502 - val_accuracy: 0.8571
Epoch 736/1000
275/275 [==============================] - 0s 86us/step - loss: 0.2322 - accuracy: 0.8582 - val_loss: 0.2507 - val_accuracy: 0.8571
Epoch 737/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2324 - accuracy: 0.8582 - val_loss: 0.2537 - val_accuracy: 0.8571
Epoch 738/1000
275/275 [==============================] - 0s 74us/step - loss: 0.2325 - accuracy: 0.8582 - val_loss: 0.2554 - val_accuracy: 0.8571
Epoch 739/1000
275/275 [==============================] - 0s 75us/step - loss: 0.2330 - accuracy: 0.8582 - val_loss: 0.2548 - val_accuracy: 0.8571
Epoch 740/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2324 - accuracy: 0.8582 - val_loss: 0.2519 - val_accuracy: 0.8571
Epoch 741/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2327 - accuracy: 0.8582 - val_loss: 0.2536 - val_acc

275/275 [==============================] - 0s 116us/step - loss: 0.2331 - accuracy: 0.8582 - val_loss: 0.2580 - val_accuracy: 0.8571
Epoch 791/1000
275/275 [==============================] - 0s 81us/step - loss: 0.2322 - accuracy: 0.8582 - val_loss: 0.2582 - val_accuracy: 0.8571
Epoch 792/1000
275/275 [==============================] - 0s 82us/step - loss: 0.2321 - accuracy: 0.8582 - val_loss: 0.2510 - val_accuracy: 0.8571
Epoch 793/1000
275/275 [==============================] - 0s 87us/step - loss: 0.2316 - accuracy: 0.8582 - val_loss: 0.2499 - val_accuracy: 0.8571
Epoch 794/1000
275/275 [==============================] - 0s 92us/step - loss: 0.2322 - accuracy: 0.8582 - val_loss: 0.2539 - val_accuracy: 0.8571
Epoch 795/1000
275/275 [==============================] - 0s 130us/step - loss: 0.2320 - accuracy: 0.8582 - val_loss: 0.2523 - val_accuracy: 0.8571
Epoch 796/1000
275/275 [==============================] - 0s 154us/step - loss: 0.2320 - accuracy: 0.8582 - val_loss: 0.2510 - val_

275/275 [==============================] - 0s 130us/step - loss: 0.2332 - accuracy: 0.8582 - val_loss: 0.2514 - val_accuracy: 0.8571
Epoch 846/1000
275/275 [==============================] - 0s 135us/step - loss: 0.2325 - accuracy: 0.8582 - val_loss: 0.2507 - val_accuracy: 0.8571
Epoch 847/1000
275/275 [==============================] - 0s 118us/step - loss: 0.2321 - accuracy: 0.8582 - val_loss: 0.2499 - val_accuracy: 0.8571
Epoch 848/1000
275/275 [==============================] - 0s 101us/step - loss: 0.2323 - accuracy: 0.8582 - val_loss: 0.2518 - val_accuracy: 0.8571
Epoch 849/1000
275/275 [==============================] - 0s 82us/step - loss: 0.2320 - accuracy: 0.8582 - val_loss: 0.2508 - val_accuracy: 0.8571
Epoch 850/1000
275/275 [==============================] - 0s 87us/step - loss: 0.2315 - accuracy: 0.8582 - val_loss: 0.2505 - val_accuracy: 0.8571
Epoch 851/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2313 - accuracy: 0.8582 - val_loss: 0.2498 - val

275/275 [==============================] - 0s 121us/step - loss: 0.2325 - accuracy: 0.8582 - val_loss: 0.2533 - val_accuracy: 0.8571
Epoch 901/1000
275/275 [==============================] - 0s 113us/step - loss: 0.2319 - accuracy: 0.8582 - val_loss: 0.2523 - val_accuracy: 0.8571
Epoch 902/1000
275/275 [==============================] - 0s 125us/step - loss: 0.2316 - accuracy: 0.8582 - val_loss: 0.2554 - val_accuracy: 0.8571
Epoch 903/1000
275/275 [==============================] - 0s 111us/step - loss: 0.2315 - accuracy: 0.8582 - val_loss: 0.2505 - val_accuracy: 0.8571
Epoch 904/1000
275/275 [==============================] - 0s 124us/step - loss: 0.2308 - accuracy: 0.8582 - val_loss: 0.2501 - val_accuracy: 0.8571
Epoch 905/1000
275/275 [==============================] - 0s 162us/step - loss: 0.2309 - accuracy: 0.8582 - val_loss: 0.2503 - val_accuracy: 0.8571
Epoch 906/1000
275/275 [==============================] - 0s 151us/step - loss: 0.2305 - accuracy: 0.8582 - val_loss: 0.2503 - 

275/275 [==============================] - 0s 207us/step - loss: 0.2301 - accuracy: 0.8582 - val_loss: 0.2485 - val_accuracy: 0.8487
Epoch 956/1000
275/275 [==============================] - 0s 181us/step - loss: 0.2297 - accuracy: 0.8582 - val_loss: 0.2490 - val_accuracy: 0.8487
Epoch 957/1000
275/275 [==============================] - 0s 154us/step - loss: 0.2291 - accuracy: 0.8582 - val_loss: 0.2473 - val_accuracy: 0.8487
Epoch 958/1000
275/275 [==============================] - 0s 157us/step - loss: 0.2301 - accuracy: 0.8582 - val_loss: 0.2507 - val_accuracy: 0.8571
Epoch 959/1000
275/275 [==============================] - 0s 128us/step - loss: 0.2297 - accuracy: 0.8582 - val_loss: 0.2483 - val_accuracy: 0.8487
Epoch 960/1000
275/275 [==============================] - 0s 162us/step - loss: 0.2298 - accuracy: 0.8582 - val_loss: 0.2485 - val_accuracy: 0.8487
Epoch 961/1000
275/275 [==============================] - 0s 148us/step - loss: 0.2294 - accuracy: 0.8582 - val_loss: 0.2491 - 

In [24]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

119/119 [==============================] - 0s 71us/step
combination test accuracy: 84.87%


In [25]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [26]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 275 samples, validate on 119 samples
Epoch 1/1000
275/275 [==============================] - 0s 990us/step - loss: 7.4387 - accuracy: 0.4509 - val_loss: 2.0350 - val_accuracy: 0.5210
Epoch 2/1000
275/275 [==============================] - 0s 128us/step - loss: 0.7867 - accuracy: 0.4873 - val_loss: 1.0935 - val_accuracy: 0.5210
Epoch 3/1000
275/275 [==============================] - 0s 172us/step - loss: 0.8460 - accuracy: 0.5018 - val_loss: 1.2482 - val_accuracy: 0.5630
Epoch 4/1000
275/275 [==============================] - 0s 218us/step - loss: 0.8148 - accuracy: 0.5964 - val_loss: 0.7142 - val_accuracy: 0.5294
Epoch 5/1000
275/275 [==============================] - 0s 150us/step - loss: 0.7300 - accuracy: 0.6000 - val_loss: 0.6937 - val_accuracy: 0.5462
Epoch 6/1000
275/275 [==============================] - 0s 150us/step - loss: 0.7178 - accuracy: 0.5600 - val_loss: 0.7005 - val_accuracy: 0.6891
Epoch 7/1000
275/275 [==============================] - 0s 145us/step - loss: 

275/275 [==============================] - 0s 81us/step - loss: 0.4550 - accuracy: 0.8364 - val_loss: 0.4599 - val_accuracy: 0.8235
Epoch 57/1000
275/275 [==============================] - 0s 75us/step - loss: 0.4466 - accuracy: 0.8436 - val_loss: 0.4528 - val_accuracy: 0.8319
Epoch 58/1000
275/275 [==============================] - 0s 73us/step - loss: 0.4415 - accuracy: 0.8255 - val_loss: 0.4488 - val_accuracy: 0.8151
Epoch 59/1000
275/275 [==============================] - 0s 75us/step - loss: 0.4389 - accuracy: 0.8218 - val_loss: 0.4440 - val_accuracy: 0.8319
Epoch 60/1000
275/275 [==============================] - 0s 77us/step - loss: 0.4284 - accuracy: 0.8400 - val_loss: 0.4391 - val_accuracy: 0.8235
Epoch 61/1000
275/275 [==============================] - 0s 87us/step - loss: 0.4275 - accuracy: 0.8291 - val_loss: 0.4278 - val_accuracy: 0.8487
Epoch 62/1000
275/275 [==============================] - 0s 100us/step - loss: 0.4216 - accuracy: 0.8364 - val_loss: 0.4310 - val_accuracy

275/275 [==============================] - 0s 93us/step - loss: 0.3129 - accuracy: 0.8436 - val_loss: 0.3332 - val_accuracy: 0.8487
Epoch 113/1000
275/275 [==============================] - 0s 88us/step - loss: 0.3065 - accuracy: 0.8436 - val_loss: 0.3268 - val_accuracy: 0.8487
Epoch 114/1000
275/275 [==============================] - 0s 85us/step - loss: 0.3235 - accuracy: 0.8400 - val_loss: 0.3452 - val_accuracy: 0.8235
Epoch 115/1000
275/275 [==============================] - 0s 89us/step - loss: 0.3125 - accuracy: 0.8436 - val_loss: 0.3311 - val_accuracy: 0.8487
Epoch 116/1000
275/275 [==============================] - 0s 82us/step - loss: 0.3052 - accuracy: 0.8545 - val_loss: 0.3263 - val_accuracy: 0.8487
Epoch 117/1000
275/275 [==============================] - 0s 88us/step - loss: 0.3376 - accuracy: 0.8327 - val_loss: 0.3277 - val_accuracy: 0.8487
Epoch 118/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2936 - accuracy: 0.8582 - val_loss: 0.3260 - val_acc

275/275 [==============================] - 0s 87us/step - loss: 0.2508 - accuracy: 0.8582 - val_loss: 0.2786 - val_accuracy: 0.8487
Epoch 168/1000
275/275 [==============================] - 0s 83us/step - loss: 0.2495 - accuracy: 0.8582 - val_loss: 0.2774 - val_accuracy: 0.8487
Epoch 169/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2485 - accuracy: 0.8582 - val_loss: 0.2744 - val_accuracy: 0.8487
Epoch 170/1000
275/275 [==============================] - 0s 94us/step - loss: 0.2469 - accuracy: 0.8582 - val_loss: 0.2790 - val_accuracy: 0.8487
Epoch 171/1000
275/275 [==============================] - 0s 80us/step - loss: 0.2469 - accuracy: 0.8582 - val_loss: 0.2800 - val_accuracy: 0.8487
Epoch 172/1000
275/275 [==============================] - 0s 75us/step - loss: 0.2467 - accuracy: 0.8582 - val_loss: 0.2727 - val_accuracy: 0.8487
Epoch 173/1000
275/275 [==============================] - 0s 80us/step - loss: 0.2522 - accuracy: 0.8545 - val_loss: 0.2755 - val_acc

Epoch 223/1000
275/275 [==============================] - 0s 90us/step - loss: 0.2385 - accuracy: 0.8582 - val_loss: 0.2750 - val_accuracy: 0.8487
Epoch 224/1000
275/275 [==============================] - 0s 75us/step - loss: 0.2388 - accuracy: 0.8582 - val_loss: 0.2689 - val_accuracy: 0.8487
Epoch 225/1000
275/275 [==============================] - 0s 76us/step - loss: 0.2360 - accuracy: 0.8582 - val_loss: 0.2642 - val_accuracy: 0.8487
Epoch 226/1000
275/275 [==============================] - 0s 80us/step - loss: 0.2354 - accuracy: 0.8582 - val_loss: 0.2658 - val_accuracy: 0.8487
Epoch 227/1000
275/275 [==============================] - 0s 74us/step - loss: 0.2359 - accuracy: 0.8582 - val_loss: 0.2689 - val_accuracy: 0.8487
Epoch 228/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2380 - accuracy: 0.8582 - val_loss: 0.2670 - val_accuracy: 0.8487
Epoch 229/1000
275/275 [==============================] - 0s 72us/step - loss: 0.2363 - accuracy: 0.8582 - val_loss: 0

...

Epoch 726/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2282 - accuracy: 0.8582 - val_loss: 0.2614 - val_accuracy: 0.8487
Epoch 727/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2294 - accuracy: 0.8582 - val_loss: 0.2622 - val_accuracy: 0.8487
Epoch 728/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2294 - accuracy: 0.8582 - val_loss: 0.2517 - val_accuracy: 0.8487
Epoch 729/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2278 - accuracy: 0.8582 - val_loss: 0.2610 - val_accuracy: 0.8487
Epoch 730/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2285 - accuracy: 0.8582 - val_loss: 0.2566 - val_accuracy: 0.8487
Epoch 731/1000
275/275 [==============================] - 0s 76us/step - loss: 0.2280 - accuracy: 0.8582 - val_loss: 0.2561 - val_accuracy: 0.8487
Epoch 732/1000
275/275 [==============================] - 0s 81us/step - loss: 0.2266 - accuracy: 0.8582 - val_loss: 0

Epoch 782/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2259 - accuracy: 0.8582 - val_loss: 0.2519 - val_accuracy: 0.8487
Epoch 783/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2269 - accuracy: 0.8582 - val_loss: 0.2611 - val_accuracy: 0.8487
Epoch 784/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2260 - accuracy: 0.8582 - val_loss: 0.2526 - val_accuracy: 0.8487
Epoch 785/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2264 - accuracy: 0.8582 - val_loss: 0.2617 - val_accuracy: 0.8487
Epoch 786/1000
275/275 [==============================] - 0s 76us/step - loss: 0.2294 - accuracy: 0.8582 - val_loss: 0.2611 - val_accuracy: 0.8487
Epoch 787/1000
275/275 [==============================] - 0s 76us/step - loss: 0.2263 - accuracy: 0.8582 - val_loss: 0.2550 - val_accuracy: 0.8487
Epoch 788/1000
275/275 [==============================] - 0s 81us/step - loss: 0.2261 - accuracy: 0.8582 - val_loss: 0

Epoch 838/1000
275/275 [==============================] - 0s 82us/step - loss: 0.2254 - accuracy: 0.8582 - val_loss: 0.2501 - val_accuracy: 0.8487
Epoch 839/1000
275/275 [==============================] - 0s 84us/step - loss: 0.2259 - accuracy: 0.8582 - val_loss: 0.2576 - val_accuracy: 0.8487
Epoch 840/1000
275/275 [==============================] - 0s 81us/step - loss: 0.2255 - accuracy: 0.8582 - val_loss: 0.2518 - val_accuracy: 0.8487
Epoch 841/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2253 - accuracy: 0.8582 - val_loss: 0.2495 - val_accuracy: 0.8487
Epoch 842/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2303 - accuracy: 0.8582 - val_loss: 0.2619 - val_accuracy: 0.8487
Epoch 843/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2303 - accuracy: 0.8582 - val_loss: 0.2613 - val_accuracy: 0.8487
Epoch 844/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2288 - accuracy: 0.8582 - val_loss: 0

Epoch 894/1000
275/275 [==============================] - 0s 82us/step - loss: 0.2311 - accuracy: 0.8582 - val_loss: 0.2618 - val_accuracy: 0.8487
Epoch 895/1000
275/275 [==============================] - 0s 80us/step - loss: 0.2311 - accuracy: 0.8582 - val_loss: 0.2617 - val_accuracy: 0.8487
Epoch 896/1000
275/275 [==============================] - 0s 80us/step - loss: 0.2310 - accuracy: 0.8582 - val_loss: 0.2615 - val_accuracy: 0.8487
Epoch 897/1000
275/275 [==============================] - 0s 79us/step - loss: 0.2298 - accuracy: 0.8582 - val_loss: 0.2615 - val_accuracy: 0.8487
Epoch 898/1000
275/275 [==============================] - 0s 74us/step - loss: 0.2278 - accuracy: 0.8582 - val_loss: 0.2614 - val_accuracy: 0.8487
Epoch 899/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2256 - accuracy: 0.8582 - val_loss: 0.2550 - val_accuracy: 0.8487
Epoch 900/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2253 - accuracy: 0.8582 - val_loss: 0

Epoch 950/1000
275/275 [==============================] - 0s 75us/step - loss: 0.2276 - accuracy: 0.8582 - val_loss: 0.2603 - val_accuracy: 0.8487
Epoch 951/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2255 - accuracy: 0.8582 - val_loss: 0.2526 - val_accuracy: 0.8487
Epoch 952/1000
275/275 [==============================] - 0s 74us/step - loss: 0.2251 - accuracy: 0.8582 - val_loss: 0.2494 - val_accuracy: 0.8487
Epoch 953/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2250 - accuracy: 0.8582 - val_loss: 0.2488 - val_accuracy: 0.8487
Epoch 954/1000
275/275 [==============================] - 0s 78us/step - loss: 0.2249 - accuracy: 0.8582 - val_loss: 0.2482 - val_accuracy: 0.8487
Epoch 955/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2346 - accuracy: 0.8582 - val_loss: 0.2602 - val_accuracy: 0.8487
Epoch 956/1000
275/275 [==============================] - 0s 77us/step - loss: 0.2258 - accuracy: 0.8582 - val_loss: 0

In [28]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

119/119 [==============================] - 0s 53us/step
combination test accuracy: 84.87%


In [29]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [30]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 323us/step - loss: 4.7103 - accuracy: 0.4773 - val_loss: 0.8119 - val_accuracy: 0.6014
Epoch 2/1000
331/331 [==============================] - 0s 69us/step - loss: 0.6968 - accuracy: 0.5287 - val_loss: 0.6674 - val_accuracy: 0.6294
Epoch 3/1000
331/331 [==============================] - 0s 82us/step - loss: 0.6642 - accuracy: 0.5770 - val_loss: 0.6966 - val_accuracy: 0.5944
Epoch 4/1000
331/331 [==============================] - 0s 73us/step - loss: 0.6554 - accuracy: 0.5921 - val_loss: 0.6386 - val_accuracy: 0.7063
Epoch 5/1000
331/331 [==============================] - 0s 77us/step - loss: 0.6409 - accuracy: 0.6163 - val_loss: 0.6403 - val_accuracy: 0.6713
Epoch 6/1000
331/331 [==============================] - 0s 82us/step - loss: 0.6247 - accuracy: 0.5982 - val_loss: 0.6647 - val_accuracy: 0.6154
Epoch 7/1000
331/331 [==============================] - 0s 80us/step - loss: 0.6211

Epoch 57/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4587 - accuracy: 0.7492 - val_loss: 0.5171 - val_accuracy: 0.6993
Epoch 58/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4482 - accuracy: 0.7734 - val_loss: 0.5365 - val_accuracy: 0.6713
Epoch 59/1000
331/331 [==============================] - 0s 73us/step - loss: 0.4445 - accuracy: 0.7764 - val_loss: 0.5172 - val_accuracy: 0.6993
Epoch 60/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4390 - accuracy: 0.7523 - val_loss: 0.5074 - val_accuracy: 0.6783
Epoch 61/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4577 - accuracy: 0.7613 - val_loss: 0.5181 - val_accuracy: 0.6993
Epoch 62/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4352 - accuracy: 0.7946 - val_loss: 0.5525 - val_accuracy: 0.6853
Epoch 63/1000
331/331 [==============================] - 0s 75us/step - loss: 0.4336 - accuracy: 0.7825 - val_loss: 0.5017 -

Epoch 113/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3708 - accuracy: 0.8278 - val_loss: 0.5117 - val_accuracy: 0.7972
Epoch 114/1000
331/331 [==============================] - 0s 93us/step - loss: 0.3734 - accuracy: 0.8218 - val_loss: 0.4169 - val_accuracy: 0.8112
Epoch 115/1000
331/331 [==============================] - 0s 89us/step - loss: 0.3664 - accuracy: 0.8369 - val_loss: 0.4163 - val_accuracy: 0.7413
Epoch 116/1000
331/331 [==============================] - 0s 92us/step - loss: 0.3899 - accuracy: 0.8036 - val_loss: 0.4421 - val_accuracy: 0.7762
Epoch 117/1000
331/331 [==============================] - 0s 93us/step - loss: 0.3725 - accuracy: 0.8248 - val_loss: 0.4478 - val_accuracy: 0.8252
Epoch 118/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3613 - accuracy: 0.8187 - val_loss: 0.4041 - val_accuracy: 0.8112
Epoch 119/1000
331/331 [==============================] - 0s 87us/step - loss: 0.3649 - accuracy: 0.8248 - val_loss: 0

Epoch 169/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4026 - accuracy: 0.7644 - val_loss: 0.4284 - val_accuracy: 0.7622
Epoch 170/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4017 - accuracy: 0.7674 - val_loss: 0.8838 - val_accuracy: 0.7343
Epoch 171/1000
331/331 [==============================] - 0s 69us/step - loss: 0.5243 - accuracy: 0.7462 - val_loss: 0.4342 - val_accuracy: 0.7483
Epoch 172/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4019 - accuracy: 0.7613 - val_loss: 0.4281 - val_accuracy: 0.7622
Epoch 173/1000
331/331 [==============================] - 0s 74us/step - loss: 0.4002 - accuracy: 0.7644 - val_loss: 0.4343 - val_accuracy: 0.7622
Epoch 174/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3987 - accuracy: 0.7644 - val_loss: 0.4439 - val_accuracy: 0.7483
Epoch 175/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4004 - accuracy: 0.7644 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3821 - accuracy: 0.7764 - val_loss: 0.4413 - val_accuracy: 0.7622
Epoch 226/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3782 - accuracy: 0.7764 - val_loss: 0.4310 - val_accuracy: 0.7622
Epoch 227/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3889 - accuracy: 0.7704 - val_loss: 0.4291 - val_accuracy: 0.7622
Epoch 228/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3807 - accuracy: 0.7855 - val_loss: 0.4202 - val_accuracy: 0.7762
Epoch 229/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3769 - accuracy: 0.8006 - val_loss: 0.4423 - val_accuracy: 0.7622
Epoch 230/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3761 - accuracy: 0.7946 - val_loss: 0.4418 - val_accuracy: 0.7622
Epoch 231/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3753 - accuracy: 0.7855 - val_loss: 0

...

Epoch 727/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3454 - accuracy: 0.8097 - val_loss: 0.4542 - val_accuracy: 0.8042
Epoch 728/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3169 - accuracy: 0.8308 - val_loss: 0.3628 - val_accuracy: 0.8042
Epoch 729/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3134 - accuracy: 0.8369 - val_loss: 0.3871 - val_accuracy: 0.8042
Epoch 730/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3251 - accuracy: 0.8338 - val_loss: 0.4816 - val_accuracy: 0.7483
Epoch 731/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3138 - accuracy: 0.8369 - val_loss: 0.3872 - val_accuracy: 0.8042
Epoch 732/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3285 - accuracy: 0.8157 - val_loss: 0.3492 - val_accuracy: 0.8042
Epoch 733/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3300 - accuracy: 0.8248 - val_loss: 0

Epoch 783/1000
331/331 [==============================] - 0s 116us/step - loss: 0.3123 - accuracy: 0.8369 - val_loss: 0.3265 - val_accuracy: 0.8392
Epoch 784/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3090 - accuracy: 0.8399 - val_loss: 0.3805 - val_accuracy: 0.8042
Epoch 785/1000
331/331 [==============================] - 0s 91us/step - loss: 0.3201 - accuracy: 0.8369 - val_loss: 0.4561 - val_accuracy: 0.7483
Epoch 786/1000
331/331 [==============================] - 0s 87us/step - loss: 0.4015 - accuracy: 0.7825 - val_loss: 0.3570 - val_accuracy: 0.7483
Epoch 787/1000
331/331 [==============================] - 0s 79us/step - loss: 0.3181 - accuracy: 0.8278 - val_loss: 0.3501 - val_accuracy: 0.8112
Epoch 788/1000
331/331 [==============================] - 0s 99us/step - loss: 0.3073 - accuracy: 0.8429 - val_loss: 0.3299 - val_accuracy: 0.8322
Epoch 789/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3059 - accuracy: 0.8429 - val_loss: 

Epoch 839/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3404 - accuracy: 0.8187 - val_loss: 0.4879 - val_accuracy: 0.7692
Epoch 840/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3407 - accuracy: 0.8157 - val_loss: 0.3521 - val_accuracy: 0.8042
Epoch 841/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3193 - accuracy: 0.8369 - val_loss: 0.3668 - val_accuracy: 0.8042
Epoch 842/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3110 - accuracy: 0.8489 - val_loss: 0.3517 - val_accuracy: 0.8042
Epoch 843/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3109 - accuracy: 0.8429 - val_loss: 0.3956 - val_accuracy: 0.8042
Epoch 844/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3376 - accuracy: 0.8187 - val_loss: 0.3676 - val_accuracy: 0.8042
Epoch 845/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3162 - accuracy: 0.8308 - val_loss: 0

Epoch 895/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3318 - accuracy: 0.8308 - val_loss: 0.3443 - val_accuracy: 0.8392
Epoch 896/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3325 - accuracy: 0.8338 - val_loss: 0.3471 - val_accuracy: 0.8252
Epoch 897/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3287 - accuracy: 0.8338 - val_loss: 0.3569 - val_accuracy: 0.7972
Epoch 898/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3270 - accuracy: 0.8369 - val_loss: 0.4355 - val_accuracy: 0.7483
Epoch 899/1000
331/331 [==============================] - 0s 71us/step - loss: 0.3352 - accuracy: 0.8248 - val_loss: 0.3464 - val_accuracy: 0.8182
Epoch 900/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3450 - accuracy: 0.8157 - val_loss: 0.3441 - val_accuracy: 0.8322
Epoch 901/1000
331/331 [==============================] - 0s 82us/step - loss: 0.3154 - accuracy: 0.8610 - val_loss: 1

Epoch 951/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3046 - accuracy: 0.8308 - val_loss: 0.3424 - val_accuracy: 0.8112
Epoch 952/1000
331/331 [==============================] - 0s 73us/step - loss: 0.2924 - accuracy: 0.8338 - val_loss: 0.3418 - val_accuracy: 0.8112
Epoch 953/1000
331/331 [==============================] - 0s 73us/step - loss: 0.2859 - accuracy: 0.8459 - val_loss: 0.2921 - val_accuracy: 0.8252
Epoch 954/1000
331/331 [==============================] - 0s 75us/step - loss: 0.2912 - accuracy: 0.8308 - val_loss: 0.2872 - val_accuracy: 0.8252
Epoch 955/1000
331/331 [==============================] - 0s 73us/step - loss: 0.2779 - accuracy: 0.8459 - val_loss: 0.3010 - val_accuracy: 0.8182
Epoch 956/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4691 - accuracy: 0.7946 - val_loss: 0.3725 - val_accuracy: 0.7762
Epoch 957/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3184 - accuracy: 0.8369 - val_loss: 0

In [32]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 69us/step
over-sampling test accuracy: 80.42%


In [33]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [34]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 367us/step - loss: 4.8441 - accuracy: 0.5408 - val_loss: 0.6873 - val_accuracy: 0.5594
Epoch 2/1000
331/331 [==============================] - 0s 54us/step - loss: 0.6695 - accuracy: 0.4985 - val_loss: 0.6758 - val_accuracy: 0.5664
Epoch 3/1000
331/331 [==============================] - 0s 59us/step - loss: 0.6631 - accuracy: 0.5378 - val_loss: 0.6642 - val_accuracy: 0.6364
Epoch 4/1000
331/331 [==============================] - 0s 58us/step - loss: 0.6537 - accuracy: 0.6435 - val_loss: 0.6708 - val_accuracy: 0.6783
Epoch 5/1000
331/331 [==============================] - 0s 60us/step - loss: 0.6478 - accuracy: 0.6616 - val_loss: 0.6780 - val_accuracy: 0.6783
Epoch 6/1000
331/331 [==============================] - 0s 63us/step - loss: 0.6440 - accuracy: 0.6495 - val_loss: 0.6547 - val_accuracy: 0.6154
Epoch 7/1000
331/331 [==============================] - 0s 59us/step - loss: 0.6391

Epoch 57/1000
331/331 [==============================] - 0s 55us/step - loss: 0.4896 - accuracy: 0.7704 - val_loss: 0.5554 - val_accuracy: 0.6993
Epoch 58/1000
331/331 [==============================] - 0s 54us/step - loss: 0.4875 - accuracy: 0.7764 - val_loss: 0.5435 - val_accuracy: 0.7063
Epoch 59/1000
331/331 [==============================] - 0s 58us/step - loss: 0.4851 - accuracy: 0.7825 - val_loss: 0.5479 - val_accuracy: 0.7063
Epoch 60/1000
331/331 [==============================] - 0s 60us/step - loss: 0.4851 - accuracy: 0.7674 - val_loss: 0.5467 - val_accuracy: 0.6923
Epoch 61/1000
331/331 [==============================] - 0s 53us/step - loss: 0.4808 - accuracy: 0.7644 - val_loss: 0.5322 - val_accuracy: 0.7133
Epoch 62/1000
331/331 [==============================] - 0s 55us/step - loss: 0.4789 - accuracy: 0.7734 - val_loss: 0.5435 - val_accuracy: 0.6923
Epoch 63/1000
331/331 [==============================] - 0s 56us/step - loss: 0.4752 - accuracy: 0.7704 - val_loss: 0.5262 -

331/331 [==============================] - 0s 58us/step - loss: 0.4217 - accuracy: 0.8127 - val_loss: 0.4604 - val_accuracy: 0.7622
Epoch 114/1000
331/331 [==============================] - 0s 59us/step - loss: 0.4028 - accuracy: 0.8066 - val_loss: 0.4881 - val_accuracy: 0.7343
Epoch 115/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3987 - accuracy: 0.7976 - val_loss: 0.4667 - val_accuracy: 0.7413
Epoch 116/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3929 - accuracy: 0.8187 - val_loss: 0.4570 - val_accuracy: 0.7622
Epoch 117/1000
331/331 [==============================] - 0s 51us/step - loss: 0.3948 - accuracy: 0.8308 - val_loss: 0.4826 - val_accuracy: 0.7762
Epoch 118/1000
331/331 [==============================] - 0s 56us/step - loss: 0.3914 - accuracy: 0.8278 - val_loss: 0.4556 - val_accuracy: 0.7483
Epoch 119/1000
331/331 [==============================] - 0s 53us/step - loss: 0.4004 - accuracy: 0.8066 - val_loss: 0.4490 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 56us/step - loss: 0.3560 - accuracy: 0.8157 - val_loss: 0.4445 - val_accuracy: 0.7832
Epoch 170/1000
331/331 [==============================] - 0s 61us/step - loss: 0.3553 - accuracy: 0.8127 - val_loss: 0.5123 - val_accuracy: 0.7413
Epoch 171/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3850 - accuracy: 0.8006 - val_loss: 0.4751 - val_accuracy: 0.8042
Epoch 172/1000
331/331 [==============================] - 0s 54us/step - loss: 0.3647 - accuracy: 0.8218 - val_loss: 0.4151 - val_accuracy: 0.7972
Epoch 173/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3545 - accuracy: 0.8218 - val_loss: 0.5072 - val_accuracy: 0.7622
Epoch 174/1000
331/331 [==============================] - 0s 54us/step - loss: 0.3607 - accuracy: 0.8218 - val_loss: 0.4210 - val_accuracy: 0.7832
Epoch 175/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3501 - accuracy: 0.8157 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 61us/step - loss: 0.3553 - accuracy: 0.8248 - val_loss: 0.4374 - val_accuracy: 0.7972
Epoch 226/1000
331/331 [==============================] - 0s 52us/step - loss: 0.3621 - accuracy: 0.8308 - val_loss: 0.4290 - val_accuracy: 0.8042
Epoch 227/1000
331/331 [==============================] - 0s 54us/step - loss: 0.3531 - accuracy: 0.8338 - val_loss: 0.4232 - val_accuracy: 0.8042
Epoch 228/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3551 - accuracy: 0.8278 - val_loss: 0.4038 - val_accuracy: 0.8182
Epoch 229/1000
331/331 [==============================] - 0s 53us/step - loss: 0.3665 - accuracy: 0.8278 - val_loss: 0.4060 - val_accuracy: 0.7902
Epoch 230/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3578 - accuracy: 0.8187 - val_loss: 0.4948 - val_accuracy: 0.7762
Epoch 231/1000
331/331 [==============================] - 0s 62us/step - loss: 0.3600 - accuracy: 0.8308 - val_loss: 0

...

Epoch 728/1000
331/331 [==============================] - 0s 56us/step - loss: 0.3764 - accuracy: 0.7704 - val_loss: 0.4094 - val_accuracy: 0.7692
Epoch 729/1000
331/331 [==============================] - 0s 59us/step - loss: 0.3759 - accuracy: 0.7704 - val_loss: 0.4098 - val_accuracy: 0.7692
Epoch 730/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3745 - accuracy: 0.7704 - val_loss: 0.4155 - val_accuracy: 0.7692
Epoch 731/1000
331/331 [==============================] - 0s 65us/step - loss: 0.3744 - accuracy: 0.7704 - val_loss: 0.4150 - val_accuracy: 0.7692
Epoch 732/1000
331/331 [==============================] - 0s 64us/step - loss: 0.3737 - accuracy: 0.7704 - val_loss: 0.4113 - val_accuracy: 0.7902
Epoch 733/1000
331/331 [==============================] - 0s 63us/step - loss: 0.3664 - accuracy: 0.7915 - val_loss: 0.4138 - val_accuracy: 0.7972
Epoch 734/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3675 - accuracy: 0.7976 - val_loss: 0

Epoch 784/1000
331/331 [==============================] - 0s 64us/step - loss: 0.3640 - accuracy: 0.7885 - val_loss: 0.4232 - val_accuracy: 0.7972
Epoch 785/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3541 - accuracy: 0.8127 - val_loss: 0.5024 - val_accuracy: 0.6573
Epoch 786/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4239 - accuracy: 0.7432 - val_loss: 0.4347 - val_accuracy: 0.7622
Epoch 787/1000
331/331 [==============================] - 0s 56us/step - loss: 0.3711 - accuracy: 0.7704 - val_loss: 0.4282 - val_accuracy: 0.7622
Epoch 788/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3681 - accuracy: 0.7915 - val_loss: 0.4051 - val_accuracy: 0.7692
Epoch 789/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3678 - accuracy: 0.8036 - val_loss: 0.4559 - val_accuracy: 0.7622
Epoch 790/1000
331/331 [==============================] - 0s 54us/step - loss: 0.3726 - accuracy: 0.7764 - val_loss: 0

Epoch 840/1000
331/331 [==============================] - 0s 60us/step - loss: 0.3562 - accuracy: 0.8036 - val_loss: 0.4551 - val_accuracy: 0.7832
Epoch 841/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3567 - accuracy: 0.8097 - val_loss: 0.4395 - val_accuracy: 0.8112
Epoch 842/1000
331/331 [==============================] - 0s 58us/step - loss: 0.3549 - accuracy: 0.8127 - val_loss: 0.4391 - val_accuracy: 0.8112
Epoch 843/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3545 - accuracy: 0.8066 - val_loss: 0.4338 - val_accuracy: 0.7902
Epoch 844/1000
331/331 [==============================] - 0s 58us/step - loss: 0.3583 - accuracy: 0.8036 - val_loss: 0.4121 - val_accuracy: 0.7902
Epoch 845/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3515 - accuracy: 0.8157 - val_loss: 0.4010 - val_accuracy: 0.7902
Epoch 846/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3509 - accuracy: 0.8157 - val_loss: 0

Epoch 896/1000
331/331 [==============================] - 0s 58us/step - loss: 0.3221 - accuracy: 0.8399 - val_loss: 0.4322 - val_accuracy: 0.8112
Epoch 897/1000
331/331 [==============================] - 0s 56us/step - loss: 0.3134 - accuracy: 0.8489 - val_loss: 0.4096 - val_accuracy: 0.7902
Epoch 898/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3372 - accuracy: 0.8308 - val_loss: 0.4767 - val_accuracy: 0.7762
Epoch 899/1000
331/331 [==============================] - 0s 55us/step - loss: 0.3429 - accuracy: 0.8248 - val_loss: 0.4572 - val_accuracy: 0.7972
Epoch 900/1000
331/331 [==============================] - 0s 62us/step - loss: 0.3332 - accuracy: 0.8429 - val_loss: 0.4341 - val_accuracy: 0.8112
Epoch 901/1000
331/331 [==============================] - 0s 58us/step - loss: 0.3373 - accuracy: 0.8187 - val_loss: 0.4013 - val_accuracy: 0.8112
Epoch 902/1000
331/331 [==============================] - 0s 59us/step - loss: 0.3146 - accuracy: 0.8459 - val_loss: 0

Epoch 952/1000
331/331 [==============================] - 0s 59us/step - loss: 0.4570 - accuracy: 0.8097 - val_loss: 0.6683 - val_accuracy: 0.7483
Epoch 953/1000
331/331 [==============================] - 0s 56us/step - loss: 0.4813 - accuracy: 0.7976 - val_loss: 0.3914 - val_accuracy: 0.8042
Epoch 954/1000
331/331 [==============================] - 0s 58us/step - loss: 0.3158 - accuracy: 0.8369 - val_loss: 0.4630 - val_accuracy: 0.7902
Epoch 955/1000
331/331 [==============================] - 0s 57us/step - loss: 0.3428 - accuracy: 0.8248 - val_loss: 0.3736 - val_accuracy: 0.8042
Epoch 956/1000
331/331 [==============================] - 0s 60us/step - loss: 0.3133 - accuracy: 0.8399 - val_loss: 0.3275 - val_accuracy: 0.8112
Epoch 957/1000
331/331 [==============================] - 0s 56us/step - loss: 0.3180 - accuracy: 0.8338 - val_loss: 0.3469 - val_accuracy: 0.8112
Epoch 958/1000
331/331 [==============================] - 0s 58us/step - loss: 0.3120 - accuracy: 0.8308 - val_loss: 0

In [36]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 61us/step
over-sampling test accuracy: 81.12%


In [37]:
############ Feature selection using lasso ##########

In [38]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [39]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [40]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [41]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [42]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 539
selected features: 28


In [43]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  2,   5,  20,  42,  43, 115, 117, 166, 173, 202, 203, 205, 206,
        209, 215, 222, 240, 246, 268, 273, 301, 323, 329, 336, 346, 372,
        537, 538]])

In [44]:
names_arr = np.array(names)
names_arr[cols]

array(['TGGGTCTGAC', 'TAGTCGCACT', 'X1_14177_G_A', 'X1_118393_A_G',
       'X1_118483_T_C', 'X1_429389_T_C', 'X1_434861_C_T', 'X1_741250_T_C',
       'X1_775214_A_G', 'group_2982', 'group_1238', 'group_2369',
       'group_1488', 'group_11116', 'hlb', 'group_2110', 'group_4324',
       'group_226', 'group_1279', 'group_1097', 'argE', 'macB',
       'group_441', 'group_8377', 'group_4766', 'group_1659', 'ST', 'CC'],
      dtype='<U100')

In [45]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   3,   4,   6,   7,   8,   9,  10,  11,
            ...
            527, 528, 529, 530, 531, 532, 533, 534, 535, 536],
           dtype='int64', length=511)

In [46]:
len(removed_features)

511

In [47]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 28) (255,) (255, 29)


In [48]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 192), (1, 180)]


In [49]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 237), (1, 237)]


In [50]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [51]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [52]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [53]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [54]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 260 samples, validate on 112 samples
Epoch 1/1000
260/260 [==============================] - 0s 430us/step - loss: 17.0314 - accuracy: 0.4962 - val_loss: 0.7061 - val_accuracy: 0.4911
Epoch 2/1000
260/260 [==============================] - 0s 72us/step - loss: 0.6867 - accuracy: 0.5846 - val_loss: 0.6789 - val_accuracy: 0.5357
Epoch 3/1000
260/260 [==============================] - 0s 75us/step - loss: 0.6512 - accuracy: 0.6808 - val_loss: 0.7163 - val_accuracy: 0.6518
Epoch 4/1000
260/260 [==============================] - 0s 79us/step - loss: 0.6305 - accuracy: 0.7346 - val_loss: 0.6515 - val_accuracy: 0.6696
Epoch 5/1000
260/260 [==============================] - 0s 81us/step - loss: 0.6106 - accuracy: 0.7500 - val_loss: 0.6422 - val_accuracy: 0.6786
Epoch 6/1000
260/260 [==============================] - 0s 87us/step - loss: 0.6063 - accuracy: 0.7385 - val_loss: 0.6631 - val_accuracy: 0.6696
Epoch 7/1000
260/260 [==============================] - 0s 81us/step - loss: 0.607

Epoch 57/1000
260/260 [==============================] - 0s 74us/step - loss: 0.4944 - accuracy: 0.7654 - val_loss: 0.5743 - val_accuracy: 0.6339
Epoch 58/1000
260/260 [==============================] - 0s 70us/step - loss: 0.4974 - accuracy: 0.7538 - val_loss: 0.5640 - val_accuracy: 0.6339
Epoch 59/1000
260/260 [==============================] - 0s 70us/step - loss: 0.4942 - accuracy: 0.7538 - val_loss: 0.5569 - val_accuracy: 0.6339
Epoch 60/1000
260/260 [==============================] - 0s 70us/step - loss: 0.4903 - accuracy: 0.7500 - val_loss: 0.5292 - val_accuracy: 0.6429
Epoch 61/1000
260/260 [==============================] - 0s 73us/step - loss: 0.4834 - accuracy: 0.7615 - val_loss: 0.5419 - val_accuracy: 0.6339
Epoch 62/1000
260/260 [==============================] - 0s 72us/step - loss: 0.4854 - accuracy: 0.7577 - val_loss: 0.6222 - val_accuracy: 0.6250
Epoch 63/1000
260/260 [==============================] - 0s 72us/step - loss: 0.5010 - accuracy: 0.7538 - val_loss: 0.5276 -

Epoch 113/1000
260/260 [==============================] - 0s 77us/step - loss: 0.4344 - accuracy: 0.8115 - val_loss: 0.5542 - val_accuracy: 0.7411
Epoch 114/1000
260/260 [==============================] - 0s 93us/step - loss: 0.4482 - accuracy: 0.7923 - val_loss: 0.4613 - val_accuracy: 0.8125
Epoch 115/1000
260/260 [==============================] - 0s 85us/step - loss: 0.4233 - accuracy: 0.8231 - val_loss: 0.4548 - val_accuracy: 0.8036
Epoch 116/1000
260/260 [==============================] - 0s 94us/step - loss: 0.4209 - accuracy: 0.8231 - val_loss: 0.4835 - val_accuracy: 0.7679
Epoch 117/1000
260/260 [==============================] - 0s 84us/step - loss: 0.4492 - accuracy: 0.7923 - val_loss: 0.4624 - val_accuracy: 0.8125
Epoch 118/1000
260/260 [==============================] - 0s 86us/step - loss: 0.4250 - accuracy: 0.8269 - val_loss: 0.5010 - val_accuracy: 0.7500
Epoch 119/1000
260/260 [==============================] - 0s 87us/step - loss: 0.4270 - accuracy: 0.8115 - val_loss: 0

Epoch 169/1000
260/260 [==============================] - 0s 82us/step - loss: 0.4052 - accuracy: 0.8231 - val_loss: 0.4277 - val_accuracy: 0.8036
Epoch 170/1000
260/260 [==============================] - 0s 70us/step - loss: 0.4026 - accuracy: 0.8423 - val_loss: 0.5602 - val_accuracy: 0.7857
Epoch 171/1000
260/260 [==============================] - 0s 71us/step - loss: 0.4297 - accuracy: 0.8231 - val_loss: 0.4431 - val_accuracy: 0.7946
Epoch 172/1000
260/260 [==============================] - 0s 76us/step - loss: 0.3992 - accuracy: 0.8346 - val_loss: 0.4246 - val_accuracy: 0.8125
Epoch 173/1000
260/260 [==============================] - 0s 71us/step - loss: 0.3975 - accuracy: 0.8423 - val_loss: 0.5650 - val_accuracy: 0.7857
Epoch 174/1000
260/260 [==============================] - 0s 73us/step - loss: 0.4098 - accuracy: 0.8269 - val_loss: 0.4260 - val_accuracy: 0.8125
Epoch 175/1000
260/260 [==============================] - 0s 71us/step - loss: 0.4049 - accuracy: 0.8500 - val_loss: 0

Epoch 225/1000
260/260 [==============================] - 0s 73us/step - loss: 0.4171 - accuracy: 0.7923 - val_loss: 0.4439 - val_accuracy: 0.7768
Epoch 226/1000
260/260 [==============================] - 0s 73us/step - loss: 0.4182 - accuracy: 0.7923 - val_loss: 0.4455 - val_accuracy: 0.7589
Epoch 227/1000
260/260 [==============================] - 0s 75us/step - loss: 0.4167 - accuracy: 0.7923 - val_loss: 0.4443 - val_accuracy: 0.7768
Epoch 228/1000
260/260 [==============================] - 0s 72us/step - loss: 0.4067 - accuracy: 0.8192 - val_loss: 0.4334 - val_accuracy: 0.7857
Epoch 229/1000
260/260 [==============================] - 0s 71us/step - loss: 0.3983 - accuracy: 0.8346 - val_loss: 0.4341 - val_accuracy: 0.7946
Epoch 230/1000
260/260 [==============================] - 0s 72us/step - loss: 0.4219 - accuracy: 0.7923 - val_loss: 0.4528 - val_accuracy: 0.7679
Epoch 231/1000
260/260 [==============================] - 0s 72us/step - loss: 0.4183 - accuracy: 0.7885 - val_loss: 0

...

Epoch 729/1000
260/260 [==============================] - 0s 83us/step - loss: 0.2669 - accuracy: 0.8538 - val_loss: 0.3434 - val_accuracy: 0.8036
Epoch 730/1000
260/260 [==============================] - 0s 73us/step - loss: 0.2613 - accuracy: 0.8577 - val_loss: 1.0548 - val_accuracy: 0.7500
Epoch 731/1000
260/260 [==============================] - 0s 81us/step - loss: 0.5886 - accuracy: 0.8115 - val_loss: 1.0242 - val_accuracy: 0.7946
Epoch 732/1000
260/260 [==============================] - 0s 76us/step - loss: 0.3674 - accuracy: 0.8577 - val_loss: 0.3812 - val_accuracy: 0.8304
Epoch 733/1000
260/260 [==============================] - 0s 73us/step - loss: 0.3473 - accuracy: 0.8615 - val_loss: 0.3732 - val_accuracy: 0.8304
Epoch 734/1000
260/260 [==============================] - 0s 74us/step - loss: 0.3454 - accuracy: 0.8615 - val_loss: 0.3812 - val_accuracy: 0.8214
Epoch 735/1000
260/260 [==============================] - 0s 74us/step - loss: 0.3426 - accuracy: 0.8615 - val_loss: 0

Epoch 785/1000
260/260 [==============================] - 0s 80us/step - loss: 0.3210 - accuracy: 0.8538 - val_loss: 0.3355 - val_accuracy: 0.8214
Epoch 786/1000
260/260 [==============================] - 0s 77us/step - loss: 0.2969 - accuracy: 0.8615 - val_loss: 0.3215 - val_accuracy: 0.8393
Epoch 787/1000
260/260 [==============================] - 0s 79us/step - loss: 0.2996 - accuracy: 0.8538 - val_loss: 0.3249 - val_accuracy: 0.8304
Epoch 788/1000
260/260 [==============================] - 0s 79us/step - loss: 0.2956 - accuracy: 0.8615 - val_loss: 0.3189 - val_accuracy: 0.8304
Epoch 789/1000
260/260 [==============================] - 0s 70us/step - loss: 0.2935 - accuracy: 0.8577 - val_loss: 0.3233 - val_accuracy: 0.8214
Epoch 790/1000
260/260 [==============================] - 0s 85us/step - loss: 0.3002 - accuracy: 0.8538 - val_loss: 0.3780 - val_accuracy: 0.8036
Epoch 791/1000
260/260 [==============================] - 0s 72us/step - loss: 0.3437 - accuracy: 0.8269 - val_loss: 0

260/260 [==============================] - 0s 77us/step - loss: 0.2957 - accuracy: 0.8538 - val_loss: 0.3082 - val_accuracy: 0.8304
Epoch 841/1000
260/260 [==============================] - 0s 87us/step - loss: 0.2863 - accuracy: 0.8615 - val_loss: 0.3077 - val_accuracy: 0.8304
Epoch 842/1000
260/260 [==============================] - 0s 89us/step - loss: 0.2904 - accuracy: 0.8577 - val_loss: 0.3273 - val_accuracy: 0.8125
Epoch 843/1000
260/260 [==============================] - 0s 74us/step - loss: 0.2966 - accuracy: 0.8538 - val_loss: 0.3198 - val_accuracy: 0.8214
Epoch 844/1000
260/260 [==============================] - 0s 77us/step - loss: 0.2951 - accuracy: 0.8577 - val_loss: 0.3147 - val_accuracy: 0.8304
Epoch 845/1000
260/260 [==============================] - 0s 72us/step - loss: 0.2895 - accuracy: 0.8615 - val_loss: 0.6847 - val_accuracy: 0.7857
Epoch 846/1000
260/260 [==============================] - 0s 78us/step - loss: 0.3493 - accuracy: 0.8423 - val_loss: 0.3101 - val_acc

260/260 [==============================] - 0s 74us/step - loss: 0.2837 - accuracy: 0.8577 - val_loss: 0.3002 - val_accuracy: 0.8393
Epoch 896/1000
260/260 [==============================] - 0s 76us/step - loss: 0.2815 - accuracy: 0.8615 - val_loss: 0.2951 - val_accuracy: 0.8393
Epoch 897/1000
260/260 [==============================] - 0s 70us/step - loss: 0.2789 - accuracy: 0.8615 - val_loss: 0.3270 - val_accuracy: 0.8036
Epoch 898/1000
260/260 [==============================] - 0s 90us/step - loss: 0.3097 - accuracy: 0.8423 - val_loss: 0.3275 - val_accuracy: 0.8214
Epoch 899/1000
260/260 [==============================] - 0s 75us/step - loss: 0.3009 - accuracy: 0.8500 - val_loss: 0.3230 - val_accuracy: 0.8125
Epoch 900/1000
260/260 [==============================] - 0s 72us/step - loss: 0.2999 - accuracy: 0.8500 - val_loss: 0.3090 - val_accuracy: 0.8304
Epoch 901/1000
260/260 [==============================] - 0s 76us/step - loss: 0.2844 - accuracy: 0.8615 - val_loss: 0.2988 - val_acc

Epoch 951/1000
260/260 [==============================] - 0s 71us/step - loss: 0.2757 - accuracy: 0.8615 - val_loss: 0.2928 - val_accuracy: 0.8393
Epoch 952/1000
260/260 [==============================] - 0s 73us/step - loss: 0.2837 - accuracy: 0.8615 - val_loss: 0.3012 - val_accuracy: 0.8393
Epoch 953/1000
260/260 [==============================] - 0s 74us/step - loss: 0.2801 - accuracy: 0.8615 - val_loss: 0.2980 - val_accuracy: 0.8393
Epoch 954/1000
260/260 [==============================] - 0s 69us/step - loss: 0.2788 - accuracy: 0.8615 - val_loss: 0.2972 - val_accuracy: 0.8304
Epoch 955/1000
260/260 [==============================] - 0s 70us/step - loss: 0.2759 - accuracy: 0.8615 - val_loss: 0.2938 - val_accuracy: 0.8393
Epoch 956/1000
260/260 [==============================] - 0s 74us/step - loss: 0.2759 - accuracy: 0.8615 - val_loss: 0.2943 - val_accuracy: 0.8304
Epoch 957/1000
260/260 [==============================] - 0s 72us/step - loss: 0.2783 - accuracy: 0.8615 - val_loss: 0

In [55]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


112/112 [==============================] - 0s 83us/step
combination test accuracy: 83.93%


In [56]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [57]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 260 samples, validate on 112 samples
Epoch 1/1000
260/260 [==============================] - 0s 750us/step - loss: 7.9401 - accuracy: 0.6731 - val_loss: 0.8313 - val_accuracy: 0.5982
Epoch 2/1000
260/260 [==============================] - 0s 71us/step - loss: 1.6551 - accuracy: 0.7346 - val_loss: 0.7854 - val_accuracy: 0.5804
Epoch 3/1000
260/260 [==============================] - 0s 83us/step - loss: 2.4393 - accuracy: 0.6731 - val_loss: 0.7399 - val_accuracy: 0.5804
Epoch 4/1000
260/260 [==============================] - 0s 90us/step - loss: 2.5741 - accuracy: 0.6462 - val_loss: 0.7039 - val_accuracy: 0.5714
Epoch 5/1000
260/260 [==============================] - 0s 89us/step - loss: 2.0741 - accuracy: 0.7000 - val_loss: 0.6884 - val_accuracy: 0.5714
Epoch 6/1000
260/260 [==============================] - 0s 91us/step - loss: 2.6363 - accuracy: 0.6692 - val_loss: 0.7066 - val_accuracy: 0.5714
Epoch 7/1000
260/260 [==============================] - 0s 90us/step - loss: 2.0062

Epoch 57/1000
260/260 [==============================] - 0s 125us/step - loss: 1.5079 - accuracy: 0.7731 - val_loss: 0.5814 - val_accuracy: 0.7232
Epoch 58/1000
260/260 [==============================] - 0s 114us/step - loss: 2.1967 - accuracy: 0.7269 - val_loss: 0.5631 - val_accuracy: 0.7232
Epoch 59/1000
260/260 [==============================] - 0s 119us/step - loss: 1.6867 - accuracy: 0.7462 - val_loss: 0.5716 - val_accuracy: 0.7321
Epoch 60/1000
260/260 [==============================] - 0s 103us/step - loss: 2.3750 - accuracy: 0.7000 - val_loss: 0.5748 - val_accuracy: 0.7411
Epoch 61/1000
260/260 [==============================] - 0s 116us/step - loss: 2.0858 - accuracy: 0.7346 - val_loss: 0.5780 - val_accuracy: 0.7232
Epoch 62/1000
260/260 [==============================] - 0s 118us/step - loss: 2.2639 - accuracy: 0.7077 - val_loss: 0.5752 - val_accuracy: 0.7232
Epoch 63/1000
260/260 [==============================] - 0s 105us/step - loss: 2.0742 - accuracy: 0.7308 - val_loss: 0

Epoch 113/1000
260/260 [==============================] - 0s 86us/step - loss: 1.6369 - accuracy: 0.7769 - val_loss: 0.5321 - val_accuracy: 0.7857
Epoch 114/1000
260/260 [==============================] - 0s 79us/step - loss: 1.8059 - accuracy: 0.7654 - val_loss: 0.5111 - val_accuracy: 0.8036
Epoch 115/1000
260/260 [==============================] - 0s 84us/step - loss: 1.8959 - accuracy: 0.7346 - val_loss: 0.5231 - val_accuracy: 0.7857
Epoch 116/1000
260/260 [==============================] - 0s 80us/step - loss: 1.9783 - accuracy: 0.7423 - val_loss: 0.5452 - val_accuracy: 0.7857
Epoch 117/1000
260/260 [==============================] - 0s 79us/step - loss: 1.8344 - accuracy: 0.7500 - val_loss: 0.5673 - val_accuracy: 0.7768
Epoch 118/1000
260/260 [==============================] - 0s 82us/step - loss: 1.9104 - accuracy: 0.7346 - val_loss: 0.5117 - val_accuracy: 0.7946
Epoch 119/1000
260/260 [==============================] - 0s 80us/step - loss: 2.3230 - accuracy: 0.7269 - val_loss: 0

Epoch 169/1000
260/260 [==============================] - 0s 84us/step - loss: 1.6520 - accuracy: 0.7346 - val_loss: 0.5193 - val_accuracy: 0.7589
Epoch 170/1000
260/260 [==============================] - 0s 83us/step - loss: 1.8347 - accuracy: 0.7154 - val_loss: 0.5197 - val_accuracy: 0.7589
Epoch 171/1000
260/260 [==============================] - 0s 82us/step - loss: 1.6054 - accuracy: 0.7308 - val_loss: 0.5250 - val_accuracy: 0.7679
Epoch 172/1000
260/260 [==============================] - 0s 82us/step - loss: 1.7483 - accuracy: 0.7462 - val_loss: 0.5234 - val_accuracy: 0.7768
Epoch 173/1000
260/260 [==============================] - 0s 83us/step - loss: 1.6532 - accuracy: 0.7346 - val_loss: 0.5207 - val_accuracy: 0.7768
Epoch 174/1000
260/260 [==============================] - 0s 80us/step - loss: 1.7512 - accuracy: 0.7462 - val_loss: 0.5181 - val_accuracy: 0.7857
Epoch 175/1000
260/260 [==============================] - 0s 87us/step - loss: 1.8168 - accuracy: 0.7308 - val_loss: 0

Epoch 225/1000
260/260 [==============================] - 0s 81us/step - loss: 1.5547 - accuracy: 0.7654 - val_loss: 0.5018 - val_accuracy: 0.7768
Epoch 226/1000
260/260 [==============================] - 0s 76us/step - loss: 1.8099 - accuracy: 0.7308 - val_loss: 0.5009 - val_accuracy: 0.7857
Epoch 227/1000
260/260 [==============================] - 0s 81us/step - loss: 1.5752 - accuracy: 0.7462 - val_loss: 0.5018 - val_accuracy: 0.7857
Epoch 228/1000
260/260 [==============================] - 0s 80us/step - loss: 1.9378 - accuracy: 0.7115 - val_loss: 0.5017 - val_accuracy: 0.7857
Epoch 229/1000
260/260 [==============================] - 0s 76us/step - loss: 2.2168 - accuracy: 0.7038 - val_loss: 0.5029 - val_accuracy: 0.7857
Epoch 230/1000
260/260 [==============================] - 0s 81us/step - loss: 1.9364 - accuracy: 0.7115 - val_loss: 0.4975 - val_accuracy: 0.7768
Epoch 231/1000
260/260 [==============================] - 0s 79us/step - loss: 1.7487 - accuracy: 0.7538 - val_loss: 0

...

Epoch 728/1000
260/260 [==============================] - 0s 84us/step - loss: 1.7450 - accuracy: 0.7231 - val_loss: 0.4877 - val_accuracy: 0.7679
Epoch 729/1000
260/260 [==============================] - 0s 80us/step - loss: 1.9617 - accuracy: 0.7192 - val_loss: 0.4885 - val_accuracy: 0.7679
Epoch 730/1000
260/260 [==============================] - 0s 85us/step - loss: 2.0127 - accuracy: 0.7231 - val_loss: 0.4873 - val_accuracy: 0.7679
Epoch 731/1000
260/260 [==============================] - 0s 82us/step - loss: 1.9851 - accuracy: 0.7000 - val_loss: 0.4879 - val_accuracy: 0.7679
Epoch 732/1000
260/260 [==============================] - 0s 84us/step - loss: 2.3674 - accuracy: 0.6923 - val_loss: 0.4883 - val_accuracy: 0.7679
Epoch 733/1000
260/260 [==============================] - 0s 83us/step - loss: 1.7375 - accuracy: 0.7308 - val_loss: 0.4885 - val_accuracy: 0.7679
Epoch 734/1000
260/260 [==============================] - 0s 80us/step - loss: 2.4215 - accuracy: 0.6923 - val_loss: 0

Epoch 784/1000
260/260 [==============================] - 0s 81us/step - loss: 1.6758 - accuracy: 0.7346 - val_loss: 0.4797 - val_accuracy: 0.7679
Epoch 785/1000
260/260 [==============================] - 0s 79us/step - loss: 2.0032 - accuracy: 0.7269 - val_loss: 0.4804 - val_accuracy: 0.7679
Epoch 786/1000
260/260 [==============================] - 0s 82us/step - loss: 2.0644 - accuracy: 0.7231 - val_loss: 0.4792 - val_accuracy: 0.7679
Epoch 787/1000
260/260 [==============================] - 0s 83us/step - loss: 1.3170 - accuracy: 0.7654 - val_loss: 0.4832 - val_accuracy: 0.7679
Epoch 788/1000
260/260 [==============================] - 0s 83us/step - loss: 1.9714 - accuracy: 0.7077 - val_loss: 0.4824 - val_accuracy: 0.7679
Epoch 789/1000
260/260 [==============================] - 0s 80us/step - loss: 2.1196 - accuracy: 0.7231 - val_loss: 0.4799 - val_accuracy: 0.7679
Epoch 790/1000
260/260 [==============================] - 0s 77us/step - loss: 1.6291 - accuracy: 0.7269 - val_loss: 0

Epoch 840/1000
260/260 [==============================] - 0s 81us/step - loss: 1.6235 - accuracy: 0.7346 - val_loss: 0.4850 - val_accuracy: 0.7679
Epoch 841/1000
260/260 [==============================] - 0s 80us/step - loss: 2.0286 - accuracy: 0.7038 - val_loss: 0.4846 - val_accuracy: 0.7679
Epoch 842/1000
260/260 [==============================] - 0s 75us/step - loss: 1.7502 - accuracy: 0.7154 - val_loss: 0.4834 - val_accuracy: 0.7679
Epoch 843/1000
260/260 [==============================] - 0s 82us/step - loss: 1.8351 - accuracy: 0.7346 - val_loss: 0.4832 - val_accuracy: 0.7679
Epoch 844/1000
260/260 [==============================] - 0s 79us/step - loss: 1.8993 - accuracy: 0.7269 - val_loss: 0.4854 - val_accuracy: 0.7679
Epoch 845/1000
260/260 [==============================] - 0s 81us/step - loss: 2.1365 - accuracy: 0.7077 - val_loss: 0.4850 - val_accuracy: 0.7679
Epoch 846/1000
260/260 [==============================] - 0s 77us/step - loss: 1.8433 - accuracy: 0.7308 - val_loss: 0

Epoch 896/1000
260/260 [==============================] - 0s 87us/step - loss: 1.6141 - accuracy: 0.7385 - val_loss: 0.4838 - val_accuracy: 0.7679
Epoch 897/1000
260/260 [==============================] - 0s 84us/step - loss: 1.6938 - accuracy: 0.7192 - val_loss: 0.4870 - val_accuracy: 0.7679
Epoch 898/1000
260/260 [==============================] - 0s 80us/step - loss: 1.5025 - accuracy: 0.7462 - val_loss: 0.4853 - val_accuracy: 0.7679
Epoch 899/1000
260/260 [==============================] - 0s 79us/step - loss: 1.9740 - accuracy: 0.7038 - val_loss: 0.4868 - val_accuracy: 0.7679
Epoch 900/1000
260/260 [==============================] - 0s 86us/step - loss: 2.1954 - accuracy: 0.7000 - val_loss: 0.4901 - val_accuracy: 0.7679
Epoch 901/1000
260/260 [==============================] - 0s 82us/step - loss: 1.9049 - accuracy: 0.7192 - val_loss: 0.4916 - val_accuracy: 0.7679
Epoch 902/1000
260/260 [==============================] - 0s 83us/step - loss: 2.0770 - accuracy: 0.7115 - val_loss: 0

Epoch 952/1000
260/260 [==============================] - 0s 78us/step - loss: 1.5278 - accuracy: 0.7692 - val_loss: 0.4797 - val_accuracy: 0.7679
Epoch 953/1000
260/260 [==============================] - 0s 85us/step - loss: 1.9967 - accuracy: 0.7346 - val_loss: 0.4773 - val_accuracy: 0.7679
Epoch 954/1000
260/260 [==============================] - 0s 80us/step - loss: 1.6665 - accuracy: 0.7423 - val_loss: 0.4792 - val_accuracy: 0.7679
Epoch 955/1000
260/260 [==============================] - 0s 77us/step - loss: 2.0753 - accuracy: 0.7115 - val_loss: 0.4799 - val_accuracy: 0.7679
Epoch 956/1000
260/260 [==============================] - 0s 81us/step - loss: 1.5810 - accuracy: 0.7692 - val_loss: 0.4772 - val_accuracy: 0.7679
Epoch 957/1000
260/260 [==============================] - 0s 81us/step - loss: 1.4990 - accuracy: 0.7462 - val_loss: 0.4794 - val_accuracy: 0.7679
Epoch 958/1000
260/260 [==============================] - 0s 85us/step - loss: 1.7849 - accuracy: 0.7346 - val_loss: 0

In [59]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

112/112 [==============================] - 0s 76us/step
combination test accuracy: 76.79%


In [60]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [61]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [62]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 318us/step - loss: 13.3307 - accuracy: 0.5740 - val_loss: 0.7007 - val_accuracy: 0.5874
Epoch 2/1000
331/331 [==============================] - 0s 61us/step - loss: 0.6612 - accuracy: 0.6012 - val_loss: 0.6655 - val_accuracy: 0.5245
Epoch 3/1000
331/331 [==============================] - 0s 68us/step - loss: 0.6441 - accuracy: 0.5801 - val_loss: 0.6538 - val_accuracy: 0.5385
Epoch 4/1000
331/331 [==============================] - 0s 74us/step - loss: 0.6534 - accuracy: 0.5680 - val_loss: 0.7719 - val_accuracy: 0.6224
Epoch 5/1000
331/331 [==============================] - 0s 73us/step - loss: 0.6575 - accuracy: 0.5891 - val_loss: 0.7553 - val_accuracy: 0.6224
Epoch 6/1000
331/331 [==============================] - 0s 75us/step - loss: 0.6411 - accuracy: 0.6012 - val_loss: 0.6597 - val_accuracy: 0.5035
Epoch 7/1000
331/331 [==============================] - 0s 61us/step - loss: 0.635

Epoch 57/1000
331/331 [==============================] - 0s 76us/step - loss: 0.5759 - accuracy: 0.6888 - val_loss: 0.7601 - val_accuracy: 0.6713
Epoch 58/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5891 - accuracy: 0.7190 - val_loss: 0.8791 - val_accuracy: 0.6014
Epoch 59/1000
331/331 [==============================] - 0s 66us/step - loss: 0.6338 - accuracy: 0.7009 - val_loss: 0.6699 - val_accuracy: 0.6713
Epoch 60/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5766 - accuracy: 0.7039 - val_loss: 0.6128 - val_accuracy: 0.6294
Epoch 61/1000
331/331 [==============================] - 0s 72us/step - loss: 0.5666 - accuracy: 0.7100 - val_loss: 0.6176 - val_accuracy: 0.6923
Epoch 62/1000
331/331 [==============================] - 0s 72us/step - loss: 0.5995 - accuracy: 0.6949 - val_loss: 0.6206 - val_accuracy: 0.6923
Epoch 63/1000
331/331 [==============================] - 0s 70us/step - loss: 0.5590 - accuracy: 0.7281 - val_loss: 0.6148 -

331/331 [==============================] - 0s 64us/step - loss: 0.5334 - accuracy: 0.7251 - val_loss: 0.5715 - val_accuracy: 0.6783
Epoch 114/1000
331/331 [==============================] - 0s 68us/step - loss: 0.5357 - accuracy: 0.7281 - val_loss: 0.5848 - val_accuracy: 0.6783
Epoch 115/1000
331/331 [==============================] - 0s 68us/step - loss: 0.5326 - accuracy: 0.7311 - val_loss: 0.6011 - val_accuracy: 0.6783
Epoch 116/1000
331/331 [==============================] - 0s 65us/step - loss: 0.5316 - accuracy: 0.7311 - val_loss: 0.6263 - val_accuracy: 0.6783
Epoch 117/1000
331/331 [==============================] - 0s 63us/step - loss: 0.5516 - accuracy: 0.7039 - val_loss: 0.6347 - val_accuracy: 0.6713
Epoch 118/1000
331/331 [==============================] - 0s 65us/step - loss: 0.5386 - accuracy: 0.7130 - val_loss: 0.5962 - val_accuracy: 0.6713
Epoch 119/1000
331/331 [==============================] - 0s 66us/step - loss: 0.5294 - accuracy: 0.7251 - val_loss: 0.5742 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 69us/step - loss: 0.5189 - accuracy: 0.7281 - val_loss: 0.5212 - val_accuracy: 0.7133
Epoch 170/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4894 - accuracy: 0.7402 - val_loss: 0.5325 - val_accuracy: 0.6783
Epoch 171/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4945 - accuracy: 0.7221 - val_loss: 0.5427 - val_accuracy: 0.6853
Epoch 172/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4844 - accuracy: 0.7432 - val_loss: 0.5278 - val_accuracy: 0.7063
Epoch 173/1000
331/331 [==============================] - 0s 66us/step - loss: 0.5254 - accuracy: 0.7039 - val_loss: 0.5194 - val_accuracy: 0.6853
Epoch 174/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4914 - accuracy: 0.7221 - val_loss: 0.5367 - val_accuracy: 0.6853
Epoch 175/1000
331/331 [==============================] - 0s 62us/step - loss: 0.4839 - accuracy: 0.7341 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4909 - accuracy: 0.7372 - val_loss: 0.5096 - val_accuracy: 0.7413
Epoch 226/1000
331/331 [==============================] - 0s 64us/step - loss: 0.5022 - accuracy: 0.7311 - val_loss: 0.5283 - val_accuracy: 0.6993
Epoch 227/1000
331/331 [==============================] - 0s 63us/step - loss: 0.5023 - accuracy: 0.7190 - val_loss: 0.5206 - val_accuracy: 0.7273
Epoch 228/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4988 - accuracy: 0.7251 - val_loss: 0.5174 - val_accuracy: 0.7273
Epoch 229/1000
331/331 [==============================] - 0s 61us/step - loss: 0.4967 - accuracy: 0.7281 - val_loss: 0.5117 - val_accuracy: 0.7343
Epoch 230/1000
331/331 [==============================] - 0s 68us/step - loss: 0.5039 - accuracy: 0.7190 - val_loss: 0.5152 - val_accuracy: 0.7273
Epoch 231/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4958 - accuracy: 0.7281 - val_loss: 0

...

Epoch 728/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4276 - accuracy: 0.7613 - val_loss: 0.4325 - val_accuracy: 0.7762
Epoch 729/1000
331/331 [==============================] - 0s 76us/step - loss: 0.4263 - accuracy: 0.7674 - val_loss: 0.4332 - val_accuracy: 0.7832
Epoch 730/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4231 - accuracy: 0.7734 - val_loss: 0.4314 - val_accuracy: 0.7692
Epoch 731/1000
331/331 [==============================] - 0s 77us/step - loss: 0.4303 - accuracy: 0.7402 - val_loss: 0.4353 - val_accuracy: 0.7692
Epoch 732/1000
331/331 [==============================] - 0s 84us/step - loss: 0.4230 - accuracy: 0.7734 - val_loss: 0.4293 - val_accuracy: 0.7692
Epoch 733/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4240 - accuracy: 0.7674 - val_loss: 0.4315 - val_accuracy: 0.7692
Epoch 734/1000
331/331 [==============================] - 0s 81us/step - loss: 0.4205 - accuracy: 0.7674 - val_loss: 0

Epoch 784/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4220 - accuracy: 0.7613 - val_loss: 0.4389 - val_accuracy: 0.7622
Epoch 785/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4216 - accuracy: 0.7613 - val_loss: 0.4392 - val_accuracy: 0.7552
Epoch 786/1000
331/331 [==============================] - 0s 59us/step - loss: 0.4222 - accuracy: 0.7523 - val_loss: 0.4364 - val_accuracy: 0.7622
Epoch 787/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4198 - accuracy: 0.7644 - val_loss: 0.4228 - val_accuracy: 0.7832
Epoch 788/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4164 - accuracy: 0.7674 - val_loss: 0.4265 - val_accuracy: 0.7832
Epoch 789/1000
331/331 [==============================] - 0s 61us/step - loss: 0.4209 - accuracy: 0.7492 - val_loss: 0.4125 - val_accuracy: 0.8042
Epoch 790/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4497 - accuracy: 0.7523 - val_loss: 0

Epoch 840/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4203 - accuracy: 0.7613 - val_loss: 0.4349 - val_accuracy: 0.7622
Epoch 841/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4198 - accuracy: 0.7613 - val_loss: 0.4344 - val_accuracy: 0.7622
Epoch 842/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4194 - accuracy: 0.7613 - val_loss: 0.4331 - val_accuracy: 0.7622
Epoch 843/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4185 - accuracy: 0.7613 - val_loss: 0.4235 - val_accuracy: 0.7762
Epoch 844/1000
331/331 [==============================] - 0s 69us/step - loss: 0.4131 - accuracy: 0.7734 - val_loss: 0.4207 - val_accuracy: 0.7832
Epoch 845/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4240 - accuracy: 0.7583 - val_loss: 0.4597 - val_accuracy: 0.7273
Epoch 846/1000
331/331 [==============================] - 0s 66us/step - loss: 0.4303 - accuracy: 0.7523 - val_loss: 0

Epoch 896/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4176 - accuracy: 0.7674 - val_loss: 0.4265 - val_accuracy: 0.7832
Epoch 897/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4163 - accuracy: 0.7704 - val_loss: 0.4233 - val_accuracy: 0.7832
Epoch 898/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4174 - accuracy: 0.7764 - val_loss: 0.4266 - val_accuracy: 0.7762
Epoch 899/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4148 - accuracy: 0.7764 - val_loss: 0.4312 - val_accuracy: 0.7552
Epoch 900/1000
331/331 [==============================] - 0s 61us/step - loss: 0.4163 - accuracy: 0.7734 - val_loss: 0.4309 - val_accuracy: 0.7622
Epoch 901/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4153 - accuracy: 0.7704 - val_loss: 0.4242 - val_accuracy: 0.7762
Epoch 902/1000
331/331 [==============================] - 0s 62us/step - loss: 0.4188 - accuracy: 0.7644 - val_loss: 0

Epoch 952/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4224 - accuracy: 0.7492 - val_loss: 0.4313 - val_accuracy: 0.7762
Epoch 953/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4170 - accuracy: 0.7674 - val_loss: 0.4269 - val_accuracy: 0.7832
Epoch 954/1000
331/331 [==============================] - 0s 60us/step - loss: 0.4162 - accuracy: 0.7704 - val_loss: 0.4349 - val_accuracy: 0.7622
Epoch 955/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4105 - accuracy: 0.7855 - val_loss: 0.4352 - val_accuracy: 0.7622
Epoch 956/1000
331/331 [==============================] - 0s 71us/step - loss: 0.4136 - accuracy: 0.7734 - val_loss: 0.4238 - val_accuracy: 0.7762
Epoch 957/1000
331/331 [==============================] - 0s 67us/step - loss: 0.4170 - accuracy: 0.7613 - val_loss: 0.4372 - val_accuracy: 0.7622
Epoch 958/1000
331/331 [==============================] - 0s 64us/step - loss: 0.4219 - accuracy: 0.7644 - val_loss: 0

In [63]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 50us/step
combination test accuracy: 75.52%


In [64]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [65]:
model2_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [66]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 403us/step - loss: 17.5494 - accuracy: 0.5076 - val_loss: 0.6965 - val_accuracy: 0.4965
Epoch 2/1000
331/331 [==============================] - 0s 64us/step - loss: 0.8135 - accuracy: 0.4502 - val_loss: 0.8468 - val_accuracy: 0.4685
Epoch 3/1000
331/331 [==============================] - 0s 74us/step - loss: 0.7092 - accuracy: 0.4048 - val_loss: 0.7045 - val_accuracy: 0.4196
Epoch 4/1000
331/331 [==============================] - 0s 72us/step - loss: 0.6918 - accuracy: 0.4109 - val_loss: 0.7044 - val_accuracy: 0.4196
Epoch 5/1000
331/331 [==============================] - 0s 65us/step - loss: 0.6895 - accuracy: 0.4230 - val_loss: 0.7004 - val_accuracy: 0.4615
Epoch 6/1000
331/331 [==============================] - 0s 71us/step - loss: 0.6899 - accuracy: 0.4260 - val_loss: 0.7019 - val_accuracy: 0.4895
Epoch 7/1000
331/331 [==============================] - 0s 66us/step - loss: 0.685

Epoch 57/1000
331/331 [==============================] - 0s 71us/step - loss: 0.6056 - accuracy: 0.6737 - val_loss: 0.6404 - val_accuracy: 0.6573
Epoch 58/1000
331/331 [==============================] - 0s 88us/step - loss: 0.6029 - accuracy: 0.6828 - val_loss: 0.6425 - val_accuracy: 0.6503
Epoch 59/1000
331/331 [==============================] - 0s 66us/step - loss: 0.6006 - accuracy: 0.6858 - val_loss: 0.6438 - val_accuracy: 0.6434
Epoch 60/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5981 - accuracy: 0.6798 - val_loss: 0.6557 - val_accuracy: 0.6224
Epoch 61/1000
331/331 [==============================] - 0s 73us/step - loss: 0.5969 - accuracy: 0.6737 - val_loss: 0.6557 - val_accuracy: 0.6224
Epoch 62/1000
331/331 [==============================] - 0s 102us/step - loss: 0.5942 - accuracy: 0.6707 - val_loss: 0.6399 - val_accuracy: 0.6434
Epoch 63/1000
331/331 [==============================] - 0s 119us/step - loss: 0.5913 - accuracy: 0.7039 - val_loss: 0.6646

331/331 [==============================] - 0s 71us/step - loss: 0.5168 - accuracy: 0.7704 - val_loss: 0.6160 - val_accuracy: 0.6853
Epoch 114/1000
331/331 [==============================] - 0s 78us/step - loss: 0.5121 - accuracy: 0.7704 - val_loss: 0.5976 - val_accuracy: 0.7063
Epoch 115/1000
331/331 [==============================] - 0s 67us/step - loss: 0.5277 - accuracy: 0.7492 - val_loss: 0.6372 - val_accuracy: 0.6783
Epoch 116/1000
331/331 [==============================] - 0s 87us/step - loss: 0.5175 - accuracy: 0.7462 - val_loss: 0.6298 - val_accuracy: 0.6783
Epoch 117/1000
331/331 [==============================] - 0s 75us/step - loss: 0.5223 - accuracy: 0.7432 - val_loss: 0.6172 - val_accuracy: 0.6853
Epoch 118/1000
331/331 [==============================] - 0s 71us/step - loss: 0.5162 - accuracy: 0.7553 - val_loss: 0.6050 - val_accuracy: 0.7063
Epoch 119/1000
331/331 [==============================] - 0s 73us/step - loss: 0.5041 - accuracy: 0.7855 - val_loss: 0.5765 - val_acc

Epoch 169/1000
331/331 [==============================] - 0s 67us/step - loss: 0.5924 - accuracy: 0.7221 - val_loss: 0.5677 - val_accuracy: 0.6573
Epoch 170/1000
331/331 [==============================] - 0s 75us/step - loss: 0.4958 - accuracy: 0.7221 - val_loss: 0.5614 - val_accuracy: 0.6713
Epoch 171/1000
331/331 [==============================] - 0s 65us/step - loss: 0.4918 - accuracy: 0.7311 - val_loss: 0.5600 - val_accuracy: 0.6713
Epoch 172/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4891 - accuracy: 0.7402 - val_loss: 0.5442 - val_accuracy: 0.6923
Epoch 173/1000
331/331 [==============================] - 0s 75us/step - loss: 0.6433 - accuracy: 0.7583 - val_loss: 0.6964 - val_accuracy: 0.5664
Epoch 174/1000
331/331 [==============================] - 0s 73us/step - loss: 0.6181 - accuracy: 0.6103 - val_loss: 0.6485 - val_accuracy: 0.6294
Epoch 175/1000
331/331 [==============================] - 0s 75us/step - loss: 0.5792 - accuracy: 0.6828 - val_loss: 0

Epoch 225/1000
331/331 [==============================] - 0s 70us/step - loss: 0.4638 - accuracy: 0.7825 - val_loss: 0.5338 - val_accuracy: 0.7413
Epoch 226/1000
331/331 [==============================] - 0s 63us/step - loss: 0.4656 - accuracy: 0.7825 - val_loss: 0.5059 - val_accuracy: 0.7552
Epoch 227/1000
331/331 [==============================] - 0s 68us/step - loss: 0.4397 - accuracy: 0.7915 - val_loss: 0.5283 - val_accuracy: 0.7413
Epoch 228/1000
331/331 [==============================] - 0s 74us/step - loss: 0.4457 - accuracy: 0.7915 - val_loss: 0.5045 - val_accuracy: 0.7483
Epoch 229/1000
331/331 [==============================] - ETA: 0s - loss: 0.4200 - accuracy: 0.81 - 0s 78us/step - loss: 0.4435 - accuracy: 0.7946 - val_loss: 0.5926 - val_accuracy: 0.7273
Epoch 230/1000
331/331 [==============================] - 0s 72us/step - loss: 0.4698 - accuracy: 0.7553 - val_loss: 0.5171 - val_accuracy: 0.7552
Epoch 231/1000
331/331 [==============================] - 0s 71us/step - los

...

Epoch 726/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3873 - accuracy: 0.8066 - val_loss: 0.4332 - val_accuracy: 0.7972
Epoch 727/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3863 - accuracy: 0.8097 - val_loss: 0.4367 - val_accuracy: 0.7972
Epoch 728/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3857 - accuracy: 0.8066 - val_loss: 0.4520 - val_accuracy: 0.7622
Epoch 729/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3861 - accuracy: 0.7946 - val_loss: 0.4526 - val_accuracy: 0.7692
Epoch 730/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3850 - accuracy: 0.7855 - val_loss: 0.4415 - val_accuracy: 0.7902
Epoch 731/1000
331/331 [==============================] - 0s 65us/step - loss: 0.3840 - accuracy: 0.7825 - val_loss: 0.4373 - val_accuracy: 0.7972
Epoch 732/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3827 - accuracy: 0.8127 - val_loss: 0

Epoch 782/1000
331/331 [==============================] - 0s 76us/step - loss: 0.3381 - accuracy: 0.8218 - val_loss: 0.4290 - val_accuracy: 0.7972
Epoch 783/1000
331/331 [==============================] - 0s 84us/step - loss: 0.3315 - accuracy: 0.8308 - val_loss: 0.4875 - val_accuracy: 0.7832
Epoch 784/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3436 - accuracy: 0.8248 - val_loss: 0.4408 - val_accuracy: 0.7972
Epoch 785/1000
331/331 [==============================] - 0s 65us/step - loss: 0.3435 - accuracy: 0.8127 - val_loss: 0.4254 - val_accuracy: 0.7762
Epoch 786/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3376 - accuracy: 0.8157 - val_loss: 0.5505 - val_accuracy: 0.7622
Epoch 787/1000
331/331 [==============================] - 0s 62us/step - loss: 0.3564 - accuracy: 0.7946 - val_loss: 0.4418 - val_accuracy: 0.7972
Epoch 788/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3439 - accuracy: 0.8248 - val_loss: 0

Epoch 838/1000
331/331 [==============================] - 0s 86us/step - loss: 0.3348 - accuracy: 0.8278 - val_loss: 0.4217 - val_accuracy: 0.7972
Epoch 839/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3330 - accuracy: 0.8218 - val_loss: 0.4885 - val_accuracy: 0.7832
Epoch 840/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3400 - accuracy: 0.8157 - val_loss: 0.4368 - val_accuracy: 0.7413
Epoch 841/1000
331/331 [==============================] - 0s 62us/step - loss: 0.3335 - accuracy: 0.8187 - val_loss: 0.4169 - val_accuracy: 0.7762
Epoch 842/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3282 - accuracy: 0.8278 - val_loss: 0.4566 - val_accuracy: 0.7972
Epoch 843/1000
331/331 [==============================] - 0s 66us/step - loss: 0.3291 - accuracy: 0.8308 - val_loss: 0.4357 - val_accuracy: 0.7413
Epoch 844/1000
331/331 [==============================] - 0s 64us/step - loss: 0.3359 - accuracy: 0.8157 - val_loss: 0

Epoch 894/1000
331/331 [==============================] - 0s 72us/step - loss: 0.3301 - accuracy: 0.8278 - val_loss: 0.4132 - val_accuracy: 0.7762
Epoch 895/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3267 - accuracy: 0.8248 - val_loss: 0.4043 - val_accuracy: 0.7832
Epoch 896/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3283 - accuracy: 0.8157 - val_loss: 0.4193 - val_accuracy: 0.7972
Epoch 897/1000
331/331 [==============================] - 0s 77us/step - loss: 0.3287 - accuracy: 0.8248 - val_loss: 0.4170 - val_accuracy: 0.7972
Epoch 898/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3285 - accuracy: 0.8308 - val_loss: 0.4239 - val_accuracy: 0.7972
Epoch 899/1000
331/331 [==============================] - 0s 73us/step - loss: 0.4527 - accuracy: 0.7704 - val_loss: 0.5209 - val_accuracy: 0.7483
Epoch 900/1000
331/331 [==============================] - 0s 67us/step - loss: 0.3756 - accuracy: 0.7825 - val_loss: 0

Epoch 950/1000
331/331 [==============================] - 0s 73us/step - loss: 0.3725 - accuracy: 0.8157 - val_loss: 0.4284 - val_accuracy: 0.7972
Epoch 951/1000
331/331 [==============================] - 0s 74us/step - loss: 0.3712 - accuracy: 0.8157 - val_loss: 0.4306 - val_accuracy: 0.7972
Epoch 952/1000
331/331 [==============================] - 0s 75us/step - loss: 0.3720 - accuracy: 0.8157 - val_loss: 0.4319 - val_accuracy: 0.7972
Epoch 953/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3710 - accuracy: 0.8157 - val_loss: 0.4349 - val_accuracy: 0.7972
Epoch 954/1000
331/331 [==============================] - 0s 70us/step - loss: 0.3700 - accuracy: 0.8127 - val_loss: 0.4380 - val_accuracy: 0.8042
Epoch 955/1000
331/331 [==============================] - 0s 68us/step - loss: 0.3703 - accuracy: 0.8127 - val_loss: 0.4481 - val_accuracy: 0.7972
Epoch 956/1000
331/331 [==============================] - 0s 69us/step - loss: 0.3694 - accuracy: 0.8157 - val_loss: 0

In [67]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

143/143 [==============================] - 0s 62us/step
combination test accuracy: 80.42%


In [68]:
############ Logistic Regression ############

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [70]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 98.32%


In [72]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 99.30%


In [74]:
############## Random Forest ##############

In [75]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [76]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 98.32%


In [77]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [78]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [79]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [80]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.98181818 0.98181818 1.        ]
0.9927272727272728


In [81]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98507463 1.         0.98484848 0.98484848 0.98484848]
0.9879240162822253
